In [1]:
import numpy as np
import pandas as pd

# sklearn
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV


#!pip install imbalanced-learn
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler


from commons import get_dummies, load_data

## Preprocessing

In [2]:
# load data
filename = "data/traindata.csv"
df_data = load_data(filename)
print(df_data.shape)
evaluation_filename = 'data/testdata.csv'
df_evaluation_data = load_data(evaluation_filename)
print(df_evaluation_data.shape)

(5000, 12)
(25000, 11)


In [3]:
# remove anomalies
anomalies_ids = [168, 404, 1848, 2374]
df_data = df_data[~df_data['id'].isin(anomalies_ids)]
df_data.shape

(4996, 12)

In [4]:
# feature engineering for "day" field. Do it for both training and testing data.
weekend = [5,6]
df_data['weekend'] = 0
df_data.loc[df_data['day'].isin(weekend),'weekend'] = 1
df_evaluation_data['weekend'] = 0
df_evaluation_data.loc[df_evaluation_data['day'].isin(weekend),'weekend'] = 1
df_data = get_dummies(df_data, columns = ["day"])
df_evaluation_data = get_dummies(df_evaluation_data, columns=["day"])

In [5]:
# feature engineering for "hour_start". Do it for both training and testing data.
hour_start_bins = pd.cut(df_data.hour_start, bins=8)
df_data['start_hour_bins'] = hour_start_bins
print(df_data.hour_start.nunique())
print(hour_start_bins.nunique())
df_data = get_dummies(df_data, ['start_hour_bins', 'hour_start'])

24
8


In [6]:
hour_start_bins = pd.cut(df_evaluation_data.hour_start, bins=8)
df_evaluation_data['start_hour_bins'] = hour_start_bins
print(hour_start_bins.nunique())
print(df_evaluation_data.hour_start.nunique())
df_evaluation_data = get_dummies(df_evaluation_data, ['start_hour_bins', 'hour_start'])

8
24


In [7]:
# feature engineering for dist_*, for the training data.
df_data["dist_max"] = df_data[["dist_gtfsstops_train","dist_gtfsstops_tram","dist_gtfsstops_bus"]].max(axis=1)
df_data["dist_mean"] = df_data[["dist_gtfsstops_train","dist_gtfsstops_tram","dist_gtfsstops_bus"]].mean(axis=1)

In [8]:
# For the testing data.
df_evaluation_data["dist_max"] = df_evaluation_data[["dist_gtfsstops_train","dist_gtfsstops_tram","dist_gtfsstops_bus"]].max(axis=1)
df_evaluation_data["dist_mean"] = df_evaluation_data[["dist_gtfsstops_train","dist_gtfsstops_tram","dist_gtfsstops_bus"]].mean(axis=1)

<h3>Select only the most important-significant features</h3>
The selected features derived using the random forest technique "clf_rf.feature_importances_", and a lot of expiremintation. As it an be seen concerning the "start_hour_bins", only the first and the last were selected, because their contribution-importance was indicated as high.

In [9]:
"""
categorical_features = [ 'weekend', 'day_0',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5',
       'start_hour_bins_(-0.023, 2.875]', 'start_hour_bins_(2.875, 5.75]',
       'start_hour_bins_(5.75, 8.625]', 'start_hour_bins_(8.625, 11.5]',
       'start_hour_bins_(11.5, 14.375]', 'start_hour_bins_(14.375, 17.25]',
       'start_hour_bins_(17.25, 20.125]', 'hour_start_0', 'hour_start_1',
       'hour_start_2', 'hour_start_3', 'hour_start_4', 'hour_start_5',
       'hour_start_6', 'hour_start_7', 'hour_start_8', 'hour_start_9',
       'hour_start_10', 'hour_start_11', 'hour_start_12', 'hour_start_13',
       'hour_start_14', 'hour_start_15', 'hour_start_16', 'hour_start_17',
       'hour_start_18', 'hour_start_19', 'hour_start_20', 'hour_start_21',
       'hour_start_22']
"""
categorical_features = ["start_hour_bins_(5.75, 8.625]", "start_hour_bins_(14.375, 17.25]", "weekend"]

numerical_features = ['dur_in_sec',"dist_max", "dist_mean", 'dist_gtfsstops_all', 'dist_gtfsstops_train', 'dist_gtfsstops_tram', 'dist_gtfsstops_bus']

In [10]:
# scale numerical features
scaler = StandardScaler()
df_data.loc[:,numerical_features ] = scaler.fit_transform(df_data[numerical_features])
print(df_data.shape)
df_evaluation_data.loc[:,numerical_features ] = scaler.transform(df_evaluation_data[numerical_features])
print(df_evaluation_data.shape)

(4996, 49)
(25000, 48)


In [11]:
df_features = df_data[numerical_features + categorical_features]
print(df_features.shape)
df_features_evaluation = df_evaluation_data[numerical_features + categorical_features]
print(df_features_evaluation.shape)

(4996, 10)
(25000, 10)


In [12]:
y_labels = df_data['purpose_validated']
y_labels.shape

(4996,)

In [13]:
# Create training and testing sets (split)
X_train, X_test, y_train, y_test = train_test_split(df_features, y_labels, test_size=0.1, random_state=42)
print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))

(4496, 10) 4496
(500, 10) 500


In [15]:
# Oversampling
ros = RandomOverSampler(random_state=42)
X_train, y_train = ros.fit_sample(X_train,y_train)

In [16]:
print(X_train.shape)
print(y_train.shape)

(6165, 10)
(6165,)


## Random Forest

In [17]:
#np.linspace(start=0,stop=10,num=10, dtype=int)
#np.arange(start=0,stop=1,step=0.1)
list(np.logspace(start=0,stop=2,num=10, dtype=int) + 2) + [None] # Only for illustration purposes

[3, 3, 4, 6, 9, 14, 23, 37, 61, 102, None]

In [18]:
np.unique(np.linspace(start=1,stop=25,num=100, dtype=int)) # Only for illustration purposes

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25])

<h3>Cross-Validation</h3>

In [57]:
do_hyper_param_tuning = False
# Hyper_param tuning
if do_hyper_param_tuning:
    clf = RandomForestClassifier(random_state=42)
    param_dist = {
        "max_depth": list(np.unique(np.logspace(start=1,stop=2.5,num=100, dtype=int) + 2)) + [None],
        "max_features": ['auto'],
        "min_samples_split": np.unique(np.linspace(start=2,stop=25,num=100, dtype=int)),
        "min_samples_leaf": np.unique(np.linspace(start=2,stop=25,num=100, dtype=int)),
        "bootstrap": [False],
        "criterion": ["gini"],
        "n_jobs": [4],
        "n_estimators": [20]
    }
    # run randomized search
    n_iter_search = 200
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                       n_iter=n_iter_search, cv=10, random_state=42, verbose=2)
    random_search.fit(X_train, y_train)
    print(random_search.best_params_)
    print(f"best score: {random_search.best_score_}")

Fitting 10 folds for each of 200 candidates, totalling 2000 fits
[CV] n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False, total=   1.4s
[CV] n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV]  n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=5, min_samples_leaf=12, max_features=auto, max_depth=40, criteri

...

[CV]  n_jobs=4, n_estimators=20, min_samples_split=6, min_samples_leaf=14, max_features=auto, max_depth=18, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=6, min_samples_leaf=14, max_features=auto, max_depth=18, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=6, min_samples_leaf=14, max_features=auto, max_depth=18, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=6, min_samples_leaf=14, max_features=auto, max_depth=18, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=6, min_samples_leaf=14, max_features=auto, max_depth=18, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=6, min_samples_leaf=14, max_features=auto, max_depth=18, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=6, min_samples_leaf=14, max_features=auto, max_depth=18, criteri

[CV]  n_jobs=4, n_estimators=20, min_samples_split=11, min_samples_leaf=14, max_features=auto, max_depth=35, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=11, min_samples_leaf=14, max_features=auto, max_depth=35, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=11, min_samples_leaf=14, max_features=auto, max_depth=35, criterion=gini, bootstrap=False, total=   0.4s
[CV] n_jobs=4, n_estimators=20, min_samples_split=11, min_samples_leaf=14, max_features=auto, max_depth=35, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=11, min_samples_leaf=14, max_features=auto, max_depth=35, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=11, min_samples_leaf=14, max_features=auto, max_depth=35, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=11, min_samples_leaf=14, max_features=auto, max_depth=35, 

[CV]  n_jobs=4, n_estimators=20, min_samples_split=4, min_samples_leaf=23, max_features=auto, max_depth=25, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=4, min_samples_leaf=23, max_features=auto, max_depth=25, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=4, min_samples_leaf=23, max_features=auto, max_depth=25, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=4, min_samples_leaf=23, max_features=auto, max_depth=25, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=4, min_samples_leaf=23, max_features=auto, max_depth=25, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=4, min_samples_leaf=23, max_features=auto, max_depth=25, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=4, min_samples_leaf=23, max_features=auto, max_depth=25, criteri

[CV]  n_jobs=4, n_estimators=20, min_samples_split=17, min_samples_leaf=16, max_features=auto, max_depth=95, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=9, min_samples_leaf=24, max_features=auto, max_depth=318, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=9, min_samples_leaf=24, max_features=auto, max_depth=318, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=9, min_samples_leaf=24, max_features=auto, max_depth=318, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=9, min_samples_leaf=24, max_features=auto, max_depth=318, criterion=gini, bootstrap=False, total=   0.3s
[CV] n_jobs=4, n_estimators=20, min_samples_split=9, min_samples_leaf=24, max_features=auto, max_depth=318, criterion=gini, bootstrap=False 
[CV]  n_jobs=4, n_estimators=20, min_samples_split=9, min_samples_leaf=24, max_features=auto, max_depth=318, 

[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed: 15.0min finished
/Users/taafoso2/PycharmProjects/why_do_we_move/.virtualenvs/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


score is mean accuracy. 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.score

In [ ]:
# remember to always change the number of estimators to 200

<h3>Model fitting using the best parametes</h3>

In [81]:
clf_rf = RandomForestClassifier(random_state=42)
clf_rf.set_params(n_jobs= -1,
                  n_estimators= 200,
                  min_samples_split= 6,
                  max_features= "auto",
                  max_depth= 153,
                  criterion= 'gini',
                  bootstrap= False)
clf_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=153, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

<h3>Using the trained model to predicted the labels of the test data</h3>

In [82]:
y_pred = clf_rf.predict(X_test)
y_pred_prob = clf_rf.predict_proba(X_test)

In [83]:
conf_matrix = confusion_matrix(y_test,y_pred)
print(conf_matrix)

[[ 27   6  18   4   6]
 [  5 102  14   1   9]
 [ 22  11  59   7   9]
 [  7   1  14  48   7]
 [ 12   7  10   6  88]]


[results]
    - A lot of misclassifications assined to class Errand, even though errand doesn't have that much support

In [84]:
print(classification_report(y_test,y_pred, output_dict=False)) 

              precision    recall  f1-score   support

      errand       0.37      0.44      0.40        61
        home       0.80      0.78      0.79       131
     leisure       0.51      0.55      0.53       108
        wait       0.73      0.62      0.67        77
        work       0.74      0.72      0.73       123

   micro avg       0.65      0.65      0.65       500
   macro avg       0.63      0.62      0.62       500
weighted avg       0.66      0.65      0.65       500



In [35]:
# TODO train with all data and the same hyper-params!!

In [ ]:
clf_rf.feature_importances_

## Save Submission CSV

In [ ]:
y_pred_evaluation = clf_rf.predict(df_features_evaluation)
y_pred_evaluation.shape

In [ ]:
df_evaluation_data['purpose_validated'] = y_pred_evaluation
with open("data/submission.csv", 'w') as outfile:
    outfile.write(df_evaluation_data[["id","purpose_validated"]].to_csv(index=False))
    outfile.flush